# Capacity Expansion Planning with EV Inclusion 

(Title will be revised in Final Version)

_Written by Kaarthi A Gnapathy for MAE 243 - Electric Power Systems Modelling Course, Fall 23_

### Loading Packages

In [1]:
using JuMP
using HiGHS
using Plots
using DataFrames, CSV
using Clp
using GLPK

### Loading Inputs (1)

In [2]:
#working directory **CHANGE IF WORKING FROM A DIFFERENT DIRECTORY/COMPUTER

#input path for reference electrification
inputs_path_ref = "wecc_2045_all_clean_expansion/reference_electrification_16_weeks/"

#input path for high electrification
inputs_path_high = "wecc_2045_all_clean_expansion/high_electrification_16_weeks/"

"wecc_2045_all_clean_expansion/high_electrification_16_weeks/"

In [3]:
#similar generator info will be used since only demand profiles are different between both datasets
generators = DataFrame(CSV.File(joinpath(inputs_path_ref, "Generators_data.csv")));

#ID for all generators for easy reference
G = generators.R_ID;

generators

Row,R_ID,Zone_name,Zone,Resource,Cluster,New_Build,Num_Units,Cap_size,Existing_Cap_MW,Existing_Cap_MWh,capex,Inv_cost_per_MWyr,capex_mwh,Inv_cost_per_MWhyr,Fixed_OM_cost_per_MWyr,Fixed_OM_cost_per_MWhyr,Var_OM_cost_per_MWh,Fuel,Heat_rate_MMBTU_per_MWh,heat_rate_mmbtu_mwh_iqr,heat_rate_mmbtu_mwh_std,Min_power,Ramp_Up_percentage,Ramp_Dn_percentage,Commit,Start_cost_per_MW,Start_fuel_MMBTU_per_MW,Up_time,Down_time,Charge_efficiency,Discharge_efficiency,Min_Duration,Max_Duration,THERM,VRE,MUSTRUN,STOR,HYDRO,RPS,CES
,Int64,String15,Int64,String31,String15,Int64,Int64,Float64,Float64,Float64,Float64,Float64,Float64,Int64,Int64,Int64,Float64,String31,Float64,Float64,Float64,Float64,Float64,Float64,Int64,Int64,Float64,Int64,Int64,Float64,Float64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64
1,1,CA_N,1,biomass,1,-1,2,18.4,36.72,0.0,0.0,0.0,0.0,0,85039,0,4.2,None,10.28,1.211,1.709,0.422,1.0,1.0,0,0,0.0,0,0,1.0,1.0,0,0,0,0,1,0,0,1,1
2,2,CA_N,1,biomass,2,-1,3,11.0,33.06,0.0,0.0,0.0,0.0,0,209053,0,10.4,None,25.26,13.444,7.968,0.274,1.0,1.0,0,0,0.0,0,0,1.0,1.0,0,0,0,0,1,0,0,1,1
3,3,CA_N,1,conventional_hydroelectric,1,-1,108,61.2,6607.44,13215.0,0.0,0.0,0.0,0,68272,0,0.0,None,9.12,0.0,0.009,0.0,0.083,0.083,0,0,0.0,0,0,1.0,1.0,0,0,0,0,0,0,1,0,1
4,4,CA_N,1,geothermal,1,-1,15,36.2,542.55,343.39,0.0,0.0,0.0,0,135226,0,0.0,None,9.12,0.0,0.0,0.398,1.0,1.0,0,0,0.0,0,0,1.0,1.0,0,0,1,0,1,0,0,1,1
5,5,CA_N,1,geothermal,2,-1,8,12.5,100.32,63.49,0.0,0.0,0.0,0,135226,0,0.0,None,9.12,0.0,0.0,0.336,1.0,1.0,0,0,0.0,0,0,1.0,1.0,0,0,1,0,1,0,0,1,1
6,6,CA_N,1,hydroelectric_pumped_storage,1,-1,15,41.0,615.15,97642.9,0.0,0.0,0.0,0,68272,0,0.0,None,0.0,0.0,0.0,0.033,1.0,1.0,0,0,0.0,0,0,0.866,0.866,0,0,0,0,0,1,0,0,1
7,7,CA_N,1,hydroelectric_pumped_storage,2,-1,5,235.2,1175.8,1.86635e5,0.0,0.0,0.0,0,68272,0,0.0,None,0.0,0.0,0.0,0.389,1.0,1.0,0,0,0.0,0,0,0.866,0.866,0,0,0,0,0,1,0,0,1
8,8,CA_N,1,onshore_wind_turbine,1,0,28,47.4,1327.6,0.0,0.0,0.0,0.0,0,43205,0,0.0,None,9.12,0.0,0.024,0.0,1.0,1.0,0,0,0.0,0,0,1.0,1.0,0,0,0,1,0,0,0,1,1
9,9,CA_N,1,small_hydroelectric,1,-1,130,1.8,231.4,0.0,0.0,0.0,0.0,0,97272,0,0.0,None,9.12,0.0,0.0,0.087,1.0,1.0,0,0,0.0,0,0,1.0,1.0,0,0,0,0,1,0,0,1,1


In [4]:
#reading reference demand input data
demand_inputs_ref = DataFrame(CSV.File(joinpath(inputs_path_ref, "Load_data.csv")))

#value of losr load (cost of involuntary non-served energy)
VOLL = demand_inputs_ref.Voll[1]

#set of price responsive demand (non-served energy) segments
S = convert(Array{Int64}, collect(skipmissing(demand_inputs_ref.Demand_segment)))

#creating a data frame for nse segments
nse = DataFrame(Segment = S, 
                NSE_Cost = VOLL.*collect(skipmissing(demand_inputs_ref.Cost_of_demand_curtailment_perMW)),
                NSE_Max = collect(skipmissing(demand_inputs_ref.Max_demand_curtailment)))

#set of sequential hours per sub-period
hours_per_period = convert(Int64, demand_inputs_ref.Hours_per_period[1])
#set of time sample sub-periods
P = convert(Array{Int64}, 1:demand_inputs_ref.Representative_Periods[1])
#sub-period cluster weights = number of hours represented by each sample period
W = convert(Array{Int64}, collect(skipmissing(demand_inputs_ref.Representative_Period_Weight)))
#set of all time steps
T = convert(Array{Int64}, demand_inputs_ref.Time_index)

#creating a vector of sample weights
sample_weight = zeros(Float64, size(T,1))
t=1
for p in P
    for h in 1:hours_per_period
        sample_weight[t]=W[p]/hours_per_period
        t += 1
    end
end

#set of zones
Z = convert(Array{Int64}, 1:6)

demand_ref = select(demand_inputs_ref, :Load_MW_z1, :Load_MW_z2, :Load_MW_z3, :Load_MW_z4, :Load_MW_z5, :Load_MW_z6);

#demand_ref

In [5]:
#read generator capacity factors by hour
variability = DataFrame(CSV.File(joinpath(inputs_path_ref, "Generators_variability.csv")))
#cleaning up the data by removing the first column of indexes
variability = variability[:,2:ncol(variability)];

#variability

In [6]:
#read fuels data
fuels = DataFrame(CSV.File(joinpath(inputs_path_ref, "Fuels_data.csv")));

#fuels

In [7]:
#reading network data
lines = DataFrame(CSV.File(joinpath(inputs_path_ref, "Network.csv")));
#removing the name of the line connections
lines = lines[:,2:ncol(lines)];
#fixed O&M costs for lines
lines.Line_Fixed_Cost_per_MW_yr = lines.Line_Reinforcement_Cost_per_MW_yr./20
#set of all lines

L = convert(Array{Int64}, lines.Network_lines);

#lines

In [8]:
#setting zones for easy reference
zones = ["z1", "z2", "z3", "z4", "z5", "z6"];

In [9]:
#calculating the associated variable costs for generators

generators.Var_Cost = zeros(Float64, size(G,1))
generators.CO2_Rate = zeros(Float64, size(G,1))
generators.Start_Cost = zeros(Float64, size(G,1))
generators.CO2_Per_Start = zeros(Float64, size(G,1))

for g in G
    # Variable cost ($/MWh) = variable O&M ($/MWh) + fuel cost ($/MMBtu) * heat rate (MMBtu/MWh)
    generators.Var_Cost[g] = generators.Var_OM_cost_per_MWh[g] +
        fuels[fuels.Fuel.==generators.Fuel[g],:Cost_per_MMBtu][1]*generators.Heat_rate_MMBTU_per_MWh[g]
    # CO2 emissions rate (tCO2/MWh) = fuel CO2 content (tCO2/MMBtu) * heat rate (MMBtu/MWh)
    generators.CO2_Rate[g] = fuels[fuels.Fuel.==generators.Fuel[g],:CO2_content_tons_per_MMBtu][1]*generators.Heat_rate_MMBTU_per_MWh[g]
    # Start-up cost ($/start/MW) = start up O&M cost ($/start/MW) + fuel cost ($/MMBtu) * start up fuel use (MMBtu/start/MW) 
    generators.Start_Cost[g] = generators.Start_cost_per_MW[g] +
        fuels[fuels.Fuel.==generators.Fuel[g],:Cost_per_MMBtu][1]*generators.Start_fuel_MMBTU_per_MW[g]
    # Start-up CO2 emissions (tCO2/start/MW) = fuel CO2 content (tCO2/MMBtu) * start up fuel use (MMBtu/start/MW) 
    generators.CO2_Per_Start[g] = fuels[fuels.Fuel.==generators.Fuel[g],:CO2_content_tons_per_MMBtu][1]*generators.Start_fuel_MMBTU_per_MW[g]
end


In [10]:
#Subset definition

#subset of thermal generators that are subject to unit commitment constraints
UC = intersect(generators.R_ID[generators.Commit.==1], G)

#subset of generators that are not subject to unit commitment constraints
ED = intersect(generators.R_ID[.!(generators.Commit.==1)], G)

#subset of storage resources
STOR = intersect(generators.R_ID[generators.STOR.>=1], G)

#subset of variable renewable resources
VRE = intersect(generators.R_ID[generators.VRE.==1], G)

#subset of new build generators
NEW = intersect(generators.R_ID[generators.New_Build.==1], G)

#subset of existing generators
OLD = intersect(generators.R_ID[.!(generators.New_Build.==1)], G)

#subset of RPS qualifying resources
RPS = intersect(generators.R_ID[generators.RPS.==1], G);

#subset of time steps that begin a sub period
START = 1:hours_per_period:maximum(T)

#subset of time periods that do not begin a sub period
INTERIOR = setdiff(T, START);

**Sets, Subsets, and Parameters Index** -- Easy modelling reference, will be deleted in the final model formulation

Sets \
\
G = Set of all generators \
S = Set of all non-served energy (price responsive demand) segments \
P = Set of time sample sub-periods \
W = Sub period cluster weights \
T = Set of all time steps \
Z = Set of zones \
L = Set of all transmission lines \
\
Subsets \
\
UC = Subset of G that contains all the generatos subject to unit commitment constraints \
ED = Subset of G that contains all the generators that are not subject to unit commitment constraints \
STOR = Subset of G that contains all the storage resources \
VRE = Subset of G that contains all the variable renewable generators \
NEW = Subset of G that contains all the new build generators \
OLD = Subset of G that contains all the old generators \
RPS = Subset of G that contains all the RPS qualifying generators \
\
Parameters\
\
nse = non served energy parameters (by s in S) \
generators = generation and storage parameters (by g in G) \
demand = demand parameters (by t in T) \
zones = network zones (by z in Z) \
lines = transmission lines (by l in L)

### Defining Model Function (2)

In [11]:
#creating a function for the capacity expansion model to test out different scenarios

function capacity_expansion(demand_profile, optimizer = HiGHS.Optimizer)
    CE = Model(optimizer)


    #DECISION VARIABLES

    #Capacity decision variables
    @variables(CE, begin

            #standard capacity variables
            vCAP[g in G]               >= 0 # power capacity (MW)
            vRET_CAP[g in OLD]         >= 0 # retirement power capacity (MW)
            vNEW_CAP[g in NEW]         >= 0 # new build power capacity (MW)
            
            #storage variables
            vE_CAP[g in STOR]          >= 0 # storage energy capacity (MWh)
            vRET_E_CAP[g in intersect(STOR, OLD)]   >= 0 # retirement storage energy capacity
            vNEW_E_CAP[g in intersect(STOR, NEW)]   >= 0 # new build storage energy capacity

            #transmission variables
            vT_CAP[l in L]             >= 0 # transmission capacity (MW)
            vRET_T_CAP[l in L]         >= 0 # retirement transmission capacity (MW)
            vNEW_T_CAP[l in L]         >= 0 # new build transmission capacity (MW)
    end)

    #unbounded till max_cap data is received
    #for g in NEW[generators[NEW, :Max_Cap_MW].>0]
    #    set_upper_bound(vNew_CAP[g], generators.Max_Cap_MW[g])
    #end

    #set upper bounds on transmission capacity expansion
    for l in L
        set_upper_bound(vNEW_T_CAP[l], lines.Line_Max_Reinforcement_MW[l])
    end

    #operational decision variables
    @variables(CE, begin
            vGEN[T, G]          >= 0 # power generation (MW)
            vCHARGE[T, STOR]    >= 0 # power charging (MW)
            vSOC[T, STOR]       >= 0 # energy storage state of charge (MW)
            vNSE[T, S, Z]       >= 0 # non-served energy/demand curtailment (MW)
            vFLOW[T, L]         >= 0 # transmission line flow (MW)
    end)

    #CONSTRAINTS

    #Supply Demand Balance Constraint
    @constraint(CE, cDemandBalance[t in T, z in Z], 
        sum(vGEN[t,g] for g in intersect(generators[generators.Zone.==z,:R_ID],G)) +
        sum(vNSE[t,s,z] for s in S) - 
        sum(vCHARGE[t,g] for g in intersect(generators[generators.Zone.==z,:R_ID],STOR)) -
        demand_profile[t,z] - 
        sum(lines[l,Symbol(string("z",z))] * vFLOW[t,l] for l in L) == 0
        )

    #capacitated constraints
    @constraints(CE, begin

            #max power constraint
            cMaxPower[t in T, g in G], vGEN[t,g] <= variability[t,g]*vCAP[g]
            
            #max charge constraint
            cMaxCharge[t in T, g in STOR], vCHARGE[t,g] <= vCAP[g]
            
            #max state of charge constraint
            cMaxSOC[t in T, g in STOR], vSOC[t,g] <= vE_CAP[g]
            
            #max NSE constraint
            cMaxNSE[t in T, s in S, z in Z], vNSE[t,s,z] <= nse.NSE_Max[s]*demand_profile[t,z]
            
            #max flow constraint
            cMaxFlow[t in T, l in L], vFLOW[t,l] <= vT_CAP[l]
            
            #min flow constraint
            cMinFlow[t in T, l in L], vFLOW[t,l] >= -vT_CAP[l]        
        end)

    #total capacity constraint
    @constraints(CE, begin

            #total capacity for exisiting units
            cCapOld[g in OLD], vCAP[g] == generators.Existing_Cap_MW[g] - vRET_CAP[g]

            #total capacity for new units
            cCapNew[g in NEW], vCAP[g] == vNEW_CAP[g]

            #total energy storage capacity for existing units
            cCapEnergyOld[g in intersect(STOR, OLD)], 
                vE_CAP[g] == generators.Existing_Cap_MWh[g] - vRET_E_CAP[g]

            #total energy storage capacity for new units
            cCapEnergyNew[g in intersect(STOR, NEW)], 
                vE_CAP[g] == vNEW_E_CAP[g]

            #total transmission capacity
            cTransCap[l in L], vT_CAP[l] == lines.Line_Max_Flow_MW[l] - vRET_T_CAP[l] + vNEW_T_CAP[l]            
        end)

    # time coupling constraints
    @constraints(CE, begin

            #ramp up, normal
            cRampUp[t in INTERIOR, g in G],
                vGEN[t,g] - vGEN[t-1,g] <= generators.Ramp_Up_percentage[g]*vCAP[g]

            #ramp up, sub-period wrapping
            cRampUpWrap[t in START, g in G],
                vGEN[t,g] - vGEN[t+hours_per_period-1,g] <= generators.Ramp_Up_percentage[g]*vCAP[g]

            #ramp down, normal
            cRampDown[t in INTERIOR, g in G],
                vGEN[t-1,g] - vGEN[t,g] <= generators.Ramp_Dn_percentage[g]*vCAP[g]

            #ramp down, sub-period warping
            cRampDownWrap[t in START, g in G],
                vGEN[t+hours_per_period-1,g] - vGEN[t,g] <= generators.Ramp_Dn_percentage[g]*vCAP[g]

            #storage state of charge, normal
            cSOC[t in INTERIOR, g in STOR],
                vSOC[t,g] == vSOC[t-1,g] + generators.Charge_efficiency[g]*vCHARGE[t,g] - 
                vGEN[t,g]/generators.Discharge_efficiency[g]

            #storage state of charge, sub-period warping
            cSOCWrap[t in START, g in STOR], 
                vSOC[t,g] == vSOC[t+hours_per_period-1,g] + generators.Charge_efficiency[g]*vCHARGE[t,g] - 
                vGEN[t,g]/generators.Discharge_efficiency[g]
        end)

    #OBJECTIVE FUNCTION
    
    #fixed cost for generation
    @expression(CE, eFixedCostsGeneration,
        #fixed costs for total capacity
        sum(generators.Fixed_OM_cost_per_MWyr[g]*vCAP[g] for g in G) +
        #investment costs for new capacity
        sum(generators.Inv_cost_per_MWyr[g]*vNEW_CAP[g] for g in NEW)
        )
    
    #fixed cost for storage
    @expression(CE, eFixedCostsStorage,
        #fixed costs for total storage capacity
        sum(generators.Fixed_OM_cost_per_MWhyr[g]*vE_CAP[g] for g in STOR) + 
        #investment costs for new storage energy capacity
        sum(generators.Inv_cost_per_MWhyr[g]*vNEW_E_CAP[g] for g in intersect(STOR, NEW))
        )
    
    #fixed cost for transmission
    @expression(CE, eFixedCostsTransmission,
     # Investment and fixed O&M costs for transmission lines
        sum(lines.Line_Fixed_Cost_per_MW_yr[l]*vT_CAP[l] +
            lines.Line_Reinforcement_Cost_per_MW_yr[l]*vNEW_T_CAP[l] for l in L)
        )

    #variable costs for all generators
    @expression(CE, eVariableCosts,
     # Variable costs for generation, weighted by hourly sample weight 
    sum(sample_weight[t]*generators.Var_Cost[g]*vGEN[t,g] for t in T, g in G)
        )

    #NSE costs
    @expression(CE, eNSECosts,
     # Non-served energy costs, weighted by hourly sample weight to ensure non-served energy costs estimate annual costs
    sum(sample_weight[t]*nse.NSE_Cost[s]*vNSE[t,s,z] for t in T, s in S, z in Z)
        )

    @objective(CE, Min,
    eFixedCostsGeneration + eFixedCostsStorage + eFixedCostsTransmission +
    eVariableCosts + eNSECosts
        )
    optimize!(CE)

end


capacity_expansion (generic function with 2 methods)

### Solving the Model (3)

In [ ]:
@time solution = capacity_expansion(demand_ref);

### Extracting Results (4)

In [ ]:
# Record generation capacity and energy results
generation = zeros(size(G,1))
for i in 1:size(G,1) 
    generation[i] = sum(sample_weight.*value.(solution.GEN)[:,G[i]].data) 
end


total_demand = sum(sum.(eachcol(sample_weight.*demand_ref)))

peak_demand = maximum(sum(eachcol(demand_ref)))
MWh_share = generation./total_demand.*100
cap_share = value.(solution.CAP).data./peak_demand.*100

generator_results = DataFrame(
    ID = G, 
    Resource = generators.Resource[G],
    Zone = generators.zone[G],
    Total_MW = value.(solution.CAP).data,
    Start_MW = generators.Existing_Cap_MW[G],
    Change_in_MW = value.(solution.CAP).data.-generators.Existing_Cap_MW[G],
    Percent_MW = cap_share,
    GWh = generation/1000,
    Percent_GWh = MWh_share
)

# energy storage energy capacity results (MWh)
storage_results = DataFrame(
    ID = STOR, 
    Zone = generators.zone[STOR],
    Resource = generators.Resource[STOR],
    Total_Storage_MWh = value.(solution.E_CAP).data,
    Start_Storage_MWh = generators.Existing_Cap_MWh[STOR],
    Change_in_Storage_MWh = value.(solution.E_CAP).data.-generators.Existing_Cap_MWh[STOR],
)


# transmission capacity results
transmission_results = DataFrame(
    Line = L, 
    Total_Transfer_Capacity = value.(solution.T_CAP).data,
    Start_Transfer_Capacity = lines.Line_Max_Flow_MW,
    Change_in_Transfer_Capacity = value.(solution.T_CAP).data.-lines.Line_Max_Flow_MW,
)


# non-served energy results by segment and zone
num_segments = maximum(S)
num_zones = maximum(Z)
nse_results = DataFrame(
    Segment = zeros(num_segments*num_zones),
    Zone = zeros(num_segments*num_zones),
    NSE_Price = zeros(num_segments*num_zones),
    Max_NSE_MW = zeros(num_segments*num_zones),
    Total_NSE_MWh = zeros(num_segments*num_zones),
    NSE_Percent_of_Demand = zeros(num_segments*num_zones)
)
i=1
for s in S
    for z in Z
        nse_results.Segment[i]=s
        nse_results.Zone[i]=z
        nse_results.NSE_Price[i]=nse.NSE_Cost[s]
        nse_results.Max_NSE_MW[i]=maximum(value.(solution.NSE)[:,s,z].data)
        nse_results.Total_NSE_MWh[i]=sum(sample_weight.*value.(solution.NSE)[:,s,z].data)
        nse_results.NSE_Percent_of_Demand[i]=sum(sample_weight.*value.(solution.NSE)[:,s,z].data)/total_demand*100
        i=i+1
    end
end

#cost_results (recorded in million dollars)
cost_results = DataFrame(
    Total_Costs = solution.cost/10^6,
    Fixed_Costs_Generation = value.(solution.FixedCostsGeneration)/10^6,
    Fixed_Costs_Storage = value.(solution.FixedCostsStorage)/10^6,
    Fixed_Costs_Transmission = value.(solution.FixedCostsTransmission)/10^6,
    Variable_Costs = value.(solution.VariableCosts)/10^6,
    NSE_Costs = value.(solution.NSECosts)/10^6
);

In [ ]:
generator_results

In [ ]:
storage_results

In [ ]:
transmission_results

In [ ]:
nse_results

In [ ]:
cost_results